<a href="https://colab.research.google.com/github/Santosdevbjj/analiseRiscosAtrasoObras/blob/main/Notebooks/01_exploracao_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Configuração visual
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]

# Carga dos dados (ajuste o caminho conforme sua estrutura)
df_obras = pd.read_csv('../data/raw/obras.csv')
df_atividades = pd.read_csv('../data/raw/atividades.csv')
df_fornecedores = pd.read_csv('../data/raw/fornecedores.csv')
df_suprimentos = pd.read_csv('../data/raw/suprimentos.csv')

# Unificação (Merge)
# 1. Unimos Suprimentos com Fornecedores para ver QUEM entregou o quê
df_pedidos = pd.merge(df_suprimentos, df_fornecedores, on='id_fornecedor', how='left')

# 2. Unimos com as Atividades para ver o IMPACTO no cronograma
df_analise = pd.merge(df_pedidos, df_atividades[['id_atividade', 'dias_atraso', 'status']], on='id_atividade', how='left')

# 3. Unimos com as Obras para ter o CONTEXTO (Cidade e Orçamento)
df_mestre = pd.merge(df_analise, df_obras[['id_obra', 'cidade', 'orcamento_estimado']], on='id_obra', how='left')

print(f"Dataset consolidado com {df_mestre.shape[0]} registros.")
df_mestre.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/obras.csv'

In [ ]:

# Agrupando por rating para ver a média de dias de atraso
sns.scatterplot(data=df_mestre, x='rating_confiabilidade', y='dias_atraso', hue='atrasou_entrega', palette='coolwarm')
plt.title('Correlação: Rating do Fornecedor vs. Dias de Atraso na Obra')
plt.xlabel('Rating de Confiabilidade (1-5)')
plt.ylabel('Dias de Atraso Acumulados')
plt.show()

# Insight estatístico
correlacao = df_mestre['rating_confiabilidade'].corr(df_mestre['dias_atraso'])
print(f"A correlação de Pearson entre Rating e Atraso é de: {correlacao:.2f}")

In [ ]:

# Calculando orçamento total em risco por cidade (onde há atraso)
df_risco = df_mestre[df_mestre['dias_atraso'] > 0].groupby('cidade')['orcamento_estimado'].sum().reset_index()

sns.barplot(data=df_risco.sort_values('orcamento_estimado', ascending=False), x='orcamento_estimado', y='cidade', palette='viridis')
plt.title('Volume Financeiro (Orçamento) em Obras com Atraso por Cidade')
plt.xlabel('Soma do Orçamento das Obras (R$)')
plt.show()

In [ ]:

sns.countplot(data=df_mestre[df_mestre['atrasou_entrega'] == 1], x='material', order=df_mestre['material'].value_counts().index)
plt.title('Frequência de Atrasos de Entrega por Tipo de Material')
plt.ylabel('Quantidade de Pedidos Atrasados')
plt.show()

In [ ]:

# Visualizando as colunas para o modelo
colunas_modelo = ['orcamento_estimado', 'rating_confiabilidade', 'material', 'cidade', 'atrasou_entrega']
target = 'dias_atraso'

print("Sugestão de Features para o Modelo:")
print(colunas_modelo)